In [2]:
import numpy as np
import sys
from collections import Counter
import random
import torch
import time
from utils.task_generator import Tasks_Generator
from tqdm import tqdm
from methods.simpleshot import Simpleshot
import os 
from utils.utils import save_pickle
from methods.paddle import PADDLE

file1 = "saved_embs/voxceleb1_3s/voxceleb1_3s_query_ecapa_embs.pkl"
file2 = "saved_embs/voxceleb1_3s/voxceleb1_3s_support_ecapa_embs.pkl"
#seeds = [sys.argv[3]]
enroll_dict = np.load(file1, allow_pickle=True)
test_dict = np.load(file2, allow_pickle=True)

seeds = [42, 56, 100, 24, 51]

In [3]:
def run_paddle_new(enroll_embs,enroll_labels,test_embs,test_labels,method_info):
    x_q = torch.tensor(test_embs)
    y_q = torch.tensor(test_labels).long().unsqueeze(2)
    x_s = torch.tensor(enroll_embs)
    y_s = torch.tensor(enroll_labels).long().unsqueeze(2)
    
    task_dic = {}
    task_dic['y_s'] = y_s
    task_dic['y_q'] = y_q
    task_dic['x_s'] = x_s
    task_dic['x_q'] = x_q
    
    method = PADDLE(**method_info)
    logs = method.run_task(task_dic)
    #acc_sample, _ = compute_confidence_interval(logs['acc'][:, -1])

    return logs['acc'][:,-1].tolist()

In [4]:
n_tasks = 30
seed = 42

run_acc_ind=[]
run_acc_trans_centroid=[]
run_acc_trans_l2_sum=[]
run_acc_EM = []
uniq_classes = sorted(list(set(enroll_dict['concat_labels'])))

merged_dict = {}
for key in enroll_dict.keys():
    merged_dict[key] = np.concatenate((enroll_dict[key],test_dict[key]),axis=0)
    #print(key)


task_generator = Tasks_Generator(uniq_classes=uniq_classes,
                                        n_tasks=n_tasks,
                                        n_ways=1251,
                                        n_ways_eff=5,
                                        n_query=5,
                                        k_shot=3,
                                        seed=int(seed))
    
start_sample_support = time.time()
test_embs, test_labels, test_audios,enroll_embs, enroll_labels, enroll_audios = task_generator.sampler_unified(merged_dict)

#test_embs, test_labels, test_audios = task_generator.sampler(test_dict, mode='query')
#enroll_embs, enroll_labels, enroll_audios = task_generator.sampler(enroll_dict, mode='support')
""""""
print(test_embs.shape)
print(test_labels.shape)
#print(test_audios.shape)
print(enroll_embs.shape)
print(enroll_labels.shape)
#print(enroll_audios.shape)


30it [00:56,  1.89s/it]
30it [00:00, 168.37it/s]


(30, 25, 192)
(30, 25)
(30, 3753, 192)
(30, 3753)


In [177]:
from utils.utils import CL2N_embeddings
normalize = True
enroll_embs, test_embs = CL2N_embeddings(enroll_embs,test_embs,normalize)
print(test_labels.shape)
print(enroll_labels.shape)

(30, 25)
(30, 10000)


In [178]:
eval = Simpleshot(avg="mean",backend="L2",method='inductive')
acc_list = eval.eval(enroll_embs, enroll_labels, test_embs, test_labels, test_audios)
print(acc_list)
acc = sum(acc_list)/len(acc_list)*100
print(acc)

Using SimpleShot inductive method with L2 norm backend
[1.0, 0.9599999785423279, 1.0, 1.0, 0.9599999785423279, 0.8399999737739563, 1.0, 0.8799999952316284, 1.0, 1.0, 1.0, 1.0, 0.9599999785423279, 0.9599999785423279, 0.9599999785423279, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8799999952316284, 0.9599999785423279, 0.9200000166893005, 0.9599999785423279, 0.9599999785423279, 0.9200000166893005, 0.9200000166893005, 0.8399999737739563]
96.26666605472565


In [90]:
eval = Simpleshot(avg="mean",backend="L2",method='transductive_centroid')
acc_list = eval.eval(enroll_embs, enroll_labels, test_embs, test_labels, test_audios)
print(acc_list)
acc = sum(acc_list)/len(acc_list)*100
print(acc)

(10, 75)
Using SimpleShot transductive centroid method with L2 norm backend.
torch.Size([10, 75])
torch.Size([10, 375])


AssertionError: Shape mismatch between predicted and groundtruth labels

In [179]:

from utils.paddle_utils import get_log_file,Logger
from methods.paddle import PADDLE

log_file = get_log_file(log_path='log_single_speaker10k', backbone='ecapa', dataset='voxceleb1', method='all_nq_5_kshot_2')
logger = Logger(__name__, log_file)

args={}
args['iter']=20
alphas = [i for i in range(0,75)]
for alpha in alphas:
    args['alpha'] = alpha
    
    args['maj_vote'] = False
    method_info = {'device':'cuda','log_file':log_file,'args':args}

    acc_list = run_paddle_new(enroll_embs, enroll_labels, test_embs, test_labels,method_info)
    #print(acc_list)
    acc = sum(acc_list)/len(acc_list)*100
    print(f"For alpha={alpha} acc is: {acc}%")

100%|██████████| 20/20 [00:00<00:00, 2801.24it/s]


For alpha=0 acc is: 96.26666605472565%


100%|██████████| 20/20 [00:00<00:00, 2751.72it/s]


For alpha=1 acc is: 96.39999945958455%


100%|██████████| 20/20 [00:00<00:00, 2841.48it/s]


...

For alpha=72 acc is: 14.000000208616257%


100%|██████████| 20/20 [00:00<00:00, 2812.52it/s]


For alpha=73 acc is: 14.000000208616257%


100%|██████████| 20/20 [00:00<00:00, 2720.84it/s]


For alpha=74 acc is: 14.000000208616257%
